In [ ]:
import os
! git clone https://github.com/martin-marek/parking-space-occupancy
os.chdir('parking-space-occupancy')

In [7]:
import torch, os, requests
from models.rcnn import RCNN
from utils import transforms, import_json
from utils import visualize as vis 
import torchvision

In [8]:
# create model
model = RCNN()

# load model weights
weights_path = 'weights.pt'
if not os.path.exists(weights_path):
    r = requests.get('https://storage.googleapis.com/pd-models/RCNN_128_square_gopro.pt')  
    with open(weights_path, 'wb') as f:
        f.write(r.content)
model.load_state_dict(torch.load(weights_path, map_location='cpu'))

/home/zazza/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zazza/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [ ]:
# inference for single image
""" image_path = "./dataset/data/images/ParkProva.jpg"
image = torchvision.io.read_image(image_path)

rois = transform_json.ultralytics_to_tensor('./dataset/data/bounding_boxes.json')
rois = transform_json.normalize_tensor_with_image(rois, image_path) """

# inference for all images in a folder
repo_path = "./RestPKLOTs-1/train/images"
annotation_path = './RestPKLOTs-1/train/_annotations.coco.json'

for root, _, files in os.walk(repo_path):
        for i, file in enumerate(files):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(root, file)
                image = torchvision.io.read_image(image_path)
                rois = import_json.extract_and_standardize_bboxes(annotation_path, image_path)

                ge = transforms.preprocess(image)

                with torch.no_grad():
                    class_logits = model(image, rois)
                    class_scores = class_logits.softmax(1)[:, 1]

                # to visualize (comment when dealing with more than 5 images)
                # vis.plot_ds_image(image, rois, class_scores)
                

In [11]:
# for inference on divided dataset 
repo_path = "./RestPKLOTs-1/train/images"
parking_directories = ['parking1', 'parking2', 'parking3']
annotation_path = './RestPKLOTs-1/train/_annotations.coco.json'

parking_counts = {parking: {"occupied": 0, "vacant": 0} for parking in parking_directories}

for parking in parking_directories:
        parking_path = os.path.join(repo_path, parking)
        for root, _, files in os.walk(parking_path):
            for i, file in enumerate(files):
                if file.endswith(('.jpg', '.jpeg', '.png')):
                    image_path = os.path.join(root, file)
                    image = torchvision.io.read_image(image_path)
                    rois = import_json.extract_and_standardize_bboxes(annotation_path, image_path)

                    image = transforms.preprocess(image)

                    with torch.no_grad():
                        class_logits = model(image, rois)
                        class_scores = class_logits.softmax(1)[:, 1]

                    occupied_count = (class_scores > 0.5).sum().item()
                    vacant_count = (class_scores <= 0.5).sum().item()

                    # Update counts in the dictionary
                    parking_counts[parking]["occupied"] += occupied_count
                    parking_counts[parking]["vacant"] += vacant_count
                if i == 1: break